In [1]:
import re
import math
import cupy as cp
import numpy as np

from zutils.ColorConsole import ColorCodes, clearOutput
from zutils.BaseMap import BaseMap


class Bathroom(BaseMap):   
    def __init__(self, file, rowsY, colsX):
        super().__init__()
        #set all board cells to zero
        self.setBoard(np.full((rowsY, colsX), np.int32(0)))
        self.defaultColor = ColorCodes.Red
   
        self.boardSize = np.array([rowsY, colsX])
        self.topHalfSlice = slice(0, math.floor(self.boardSize[0] / 2))
        self.botHalfSlice = slice(math.ceil(self.boardSize[0] / 2), self.boardSize[0])
        self.leftHalfSlice = slice(0, math.floor(self.boardSize[1] / 2))
        self.rightHalfSlice = slice(math.ceil(self.boardSize[1] / 2), self.boardSize[1])
        self.leftHalfReverseSlice = slice(-math.ceil(self.boardSize[1] / 2)-1, -self.boardSize[1]-1, -1)

        self.robotPositions = []
        self.robotVectors = []
        self.currentMoveNumber = 0
        # show cells without robots in White
        self.setColorMap({
            0: ColorCodes.White
        })
        self.readFile(file)
                

    def readFile(self, fileName):
        with open(fileName, 'r') as file:
            for line in file:
                splitLine = re.split("[=,\ \n]", line)
                self.addRobot([splitLine[2], splitLine[1]], [splitLine[5], splitLine[4]])
        self.robotPositions = cp.array(self.robotPositions, dtype=np.int32)
        self.robotVectors = cp.array(self.robotVectors, dtype=np.int32)


    def writeFile(self, fileName):
        outputStr = ""
        i = 0
        for i in range(len(self.robotPositions)):
            outputStr += "p={0},{1} v={2},{3}\n".format(
                self.robotPositions[i][1], self.robotPositions[i][0], 
                self.robotVectors[i][1], self.robotVectors[i][0])

        with open(fileName, 'w') as file:
            for line in outputStr:
                file.write(line)
            file.close()


    def addRobot(self, posYX, moveYX):
        self.robotPositions += [posYX]
        self.robotVectors += [moveYX]


    def moveNth(self, n):
        self.currentMoveNumber += n
        # move robot position n times (forward or back in time)
        gpu_boardSize = cp.array(self.boardSize)
        gpu_board = cp.array(self.board)
        self.robotPositions = (self.robotPositions + (n * self.robotVectors)) % gpu_boardSize

        #set all board cells to zero
        gpu_board[True, True] = 0
        # get Y and X indices of robot positions
        robotYIndices = self.robotPositions[:,0]
        robotXIndices = self.robotPositions[:,1]
        # increment array at each robot Y,X index, the same index can repeat if more than one robot is in the same cell
        cp.add.at(gpu_board, (robotYIndices, robotXIndices), 1)
        self.board = cp.asnumpy(gpu_board)
    

    def resetMoves(self):
        self.moveNth(-self.currentMoveNumber)


    def calculateSymmetryFactor(self):
        # reverse the left half columns
        leftReversedHalf = self.board[:, self.leftHalfReverseSlice].copy()
        # set left half to -1 where it's currently 0
        leftReversedHalf[leftReversedHalf != 0] = 1
        leftReversedHalf[leftReversedHalf == 0] = -1
        # get the right half as is
        rightHalf = self.board[:, self.rightHalfSlice].copy()
        rightHalf[rightHalf != 0] = 1
        # check symmetry factor between left reversed half and right half 
        symetryFactor = np.sum(leftReversedHalf == rightHalf)
        return symetryFactor

    def calcSafetyFactor(self):
        return \
            np.sum(self.board[self.topHalfSlice, self.leftHalfSlice]) * \
            np.sum(self.board[self.topHalfSlice, self.rightHalfSlice]) * \
            np.sum(self.board[self.botHalfSlice, self.leftHalfSlice]) * \
            np.sum(self.board[self.botHalfSlice, self.rightHalfSlice])


In [2]:
import time

#board = Bathroom('input/Day14-mini.txt', 7, 11)
#board = Bathroom('input/Day14.txt', 103, 101)
board = Bathroom('input/Day14-custom.txt', 75, 67)
board.moveNth(100)

cost = board.calcSafetyFactor()
print("Part 1 - safety factor: {0}".format(cost))

# move back in time 100 seconds
board.moveNth(-100)
maxSymFactor = 0
lastRepeatIter = 0
for i in range(1,10000):
    board.moveNth(1)
    symFac = board.calculateSymmetryFactor()

    if symFac > maxSymFactor:
        maxSymFactor = symFac
        clearOutput(wait=True)
        print(board)
        print("Part 1 - safety factor: {0}".format(cost))
        print("Part 2 - Iteration {0}, symmetryFactor {1}".format(i, symFac))
        time.sleep(.5)

    if symFac == maxSymFactor:
        print("Repeating - Iteration {0}, symmetryFactor {1}, repeatAfter {2}".format(i, symFac, i - lastRepeatIter))
        lastRepeatIter = i

                    1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6
0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6
______________________________________________________________________________________________________________________________________
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |1
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |2
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |3
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [3]:
from zutils.ImageUtils import convertImageToAscii

# use this function to create any random input from an image
def createRandomDay14InputFileFromImage(imageFile, outputFile, width, height):
    asciiStr = convertImageToAscii(imageFile)

    map = BaseMap()
    map.createFromString(asciiStr)

    indicesPos = np.where(map.board == '1')
    siz = len(indicesPos[1])
    vectors = np.random.randint(-100, 100, size=(siz, 2))

    outputStr = ""
    i = 0
    for vector in vectors:
        outputStr += "p={0},{1} v={2},{3}\n".format(indicesPos[1][i], indicesPos[0][i], vector[1], vector[0])
        i += 1

    with open(outputFile, 'w') as file:
        for line in outputStr:
            file.write(line)
        file.close()

    # offset initial vectors by a few positions to garble them up. Use this as the final file for distribution
    board = Bathroom(outputFile, 75, 67)
    board.moveNth(0)
    print(board)
    board.moveNth(np.random.randint(1, 1000))
    board.writeFile(outputFile)


# create a random input file from a sample image
createRandomDay14InputFileFromImage('images/cat.png', 'output/Day14-cat.txt', 75, 67)


                    1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6
0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6
______________________________________________________________________________________________________________________________________
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |1
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |2
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |3
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0